In [1]:
import pandas as pd
import numpy as np
import requests
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from pandas_profiling import ProfileReport

In [2]:
df_1 = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/1gvZ03uAL6THwd04Y98GtIj6SeAHiKyQY5UisuuyFSUs/export?format=csv', low_memory=False
)
df_2 = pd.read_csv(
    'https://docs.google.com/spreadsheets/d/1wduqo5WyYmCpaGnE81sLNGU0VSodIekMfpmEwU0fGqs/export?format=csv', low_memory=False
)
pd.options.display.max_columns = None

df_mergeado = df_2.merge(df_1, left_on = 'id', right_on = 'id')
df_mergeado.sort_values(by=['id'], inplace=True, ascending=True)


,barrio,dia,direccion_viento_tarde,direccion_viento_temprano,horas_de_sol,humedad_tarde,humedad_temprano,id,llovieron_hamburguesas_hoy,mm_evaporados_agua,mm_lluvia_dia,nubosidad_tarde,nubosidad_temprano,presion_atmosferica_tarde,presion_atmosferica_temprano,rafaga_viento_max_direccion,rafaga_viento_max_velocidad,temp_max,temp_min,temperatura_tarde,temperatura_temprano,velocidad_viendo_tarde,velocidad_viendo_temprano,llovieron_hamburguesas_al_dia_siguiente
43694,Agronomía,2008-12-01,Oestenoroeste,Oeste,NaN,22.0,71.0,0,no,NaN,0.6,NaN,8.0,1007.1,1007.7,Oeste,44.0,22.9,13.4,21.8,16.9,24.0,20.0,no
1607,Agronomía,2008-12-02,Oestesuroeste,Nornoreste,NaN,25.0,44.0,1,no,NaN,0.0,NaN,NaN,1007.8,1010.6,Oestenoroeste,44.0,25.1,7.4,24.3,17.2,22.0,4.0,no
100483,Agronomía,2008-12-03,Oestesuroeste,Oeste,NaN,30.0,38.0,2,no,NaN,0.0,2.0,NaN,1008.7,1007.6,Oestesuroeste,46.0,25.7,12.9,23.2,21.0,26.0,19.0,no
29574,Agronomía,2008-12-04,Este,Sureste,NaN,16.0,45.0,3,no,NaN,0.0,NaN,NaN,1012.8,1017.6,Noreste,24.0,28.0,9.2,26.5,18.1,9.0,11.0,no
42669,Agronomía,2008-12-05,Noroeste,Estenoreste,NaN,33.0,82.0,4,no,NaN,1.0,8.0,7.0,1006.0,1010.8,Oeste,41.0,32.3,17.5,29.7,17.8,20.0,7.0,no


In [3]:
###### df_mergeado.fillna(np.nan, inplace = True)

# Feature engineering
## Variables __categoricas__
1. Barrio 
2. direccion_viento_tarde
3. direccion_viento_temprano
4. rafaga_viento_max_direccion

### Comenzamos por reemplazar los valores nulos en direcciones de viento y ráfagas por una nueva opcion que puede tomar la feature : "Otra direccion"

In [4]:

df_mergeado['direccion_viento_tarde'] = df_mergeado['direccion_viento_tarde'].replace(np.nan, 'Otra direccion')
df_mergeado['direccion_viento_temprano'] = df_mergeado['direccion_viento_temprano'].replace(np.nan, 'Otra direccion')
df_mergeado['rafaga_viento_max_direccion'] = df_mergeado['rafaga_viento_max_direccion'].replace(np.nan, 'Otra direccion')

### Luego verificamos que ninguna feature categorica tenga valores repetidos o sin sentido por errores de tipado 

In [5]:
df_mergeado.barrio.value_counts().to_frame()

,barrio
Liniers,2737
Constitución,2657
Villa Soldati,2611
Retiro,2580
Villa Luro,2578
Villa Real,2561
Parque Avellaneda,2526
San Telmo,2518
San Nicolás,2482
Saavedra,2464


In [6]:
df_mergeado.direccion_viento_temprano.value_counts().to_frame()

,direccion_viento_temprano
Nornoreste,12784
Norte,9453
Otra direccion,8403
Sureste,7437
Este,7391
Sursureste,7244
Noroeste,7003
Sur,6948
suroeste,6785
Oeste,6742


In [7]:
df_mergeado.direccion_viento_tarde.value_counts().to_frame()

,direccion_viento_tarde
Nornoreste,11546
Sureste,8657
Oeste,8053
Sur,7930
Oestesuroeste,7627
Sursureste,7480
suroeste,7477
Norte,7180
Oestenoroeste,7096
Noroeste,6893


In [8]:
df_mergeado.llovieron_hamburguesas_hoy.value_counts().to_frame()

,llovieron_hamburguesas_hoy
no,88359
si,25394


In [9]:
df_mergeado.rafaga_viento_max_direccion.value_counts().to_frame()

,rafaga_viento_max_direccion
Nornoreste,10512
Otra direccion,8269
Oeste,7952
Sureste,7502
Norte,7432
Este,7428
Sursureste,7365
Sur,7314
Oestesuroeste,7227
suroeste,7161


#### Vemos que todos los valores son unicos, no hay necesidad de alterar las features y podemos seguir con la conversion a valores numericos